In [2]:
import requests
import json
from datetime import datetime, timedelta
from IPython.display import display, clear_output


In [ ]:
# OpenWeatherMap API Key
OPENWEATHER_API_KEY = 'YOUR_OPENWEATHERMAP_API_KEY'

In [7]:
def get_joke_data():
    url = "https://v2.jokeapi.dev/joke/Any"
    response = requests.get(url)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        print("Error fetching joke data.")
        return None
    

In [8]:
get_joke_data()

{'error': False, 'category': 'Dark', 'type': 'twopart', 'setup': "What's yellow and can't swim?", 'delivery': 'A bus full of children.', 'flags': {'nsfw': False, 'religious': False, 'political': False, 'racist': False, 'sexist': False, 'explicit': False}, 'safe': False, 'id': 310, 'lang': 'en'}


{'error': False,
 'category': 'Dark',
 'type': 'twopart',
 'setup': "What's yellow and can't swim?",
 'delivery': 'A bus full of children.',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'safe': False,
 'id': 310,
 'lang': 'en'}